<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M4_PatentSBERTa_For_PatentSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import datasets
import pandas as pd
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
from sentence_transformers import SentenceTransformer, util
from tokenizers import Tokenizer
from datetime import datetime
import random
import itertools
import pickle

In [8]:
%%time
train_dataset = datasets.load_dataset("AI-Growth-Lab/patents_claims_1.5m_traim_test", split="train[:100]")
test_dataset = datasets.load_dataset("AI-Growth-Lab/patents_claims_1.5m_traim_test", split="test[:10]")

df_claim_cpc_test = pd.DataFrame(test_dataset)
df_claim_cpc_train = pd.DataFrame(train_dataset)

CPU times: user 1.73 s, sys: 27.9 ms, total: 1.76 s
Wall time: 4.49 s


In [9]:
df_claim_cpc_train.head()

,id,date,text,A01B,A01C,A01D,A01F,A01G,A01H,A01J,...,Y02B,Y02C,Y02D,Y02E,Y02P,Y02T,Y02W,Y04S,Y10S,Y10T
0,8788730,2014-07-22,1. A method for sending a keycode of a non-key...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8621421,2013-12-31,1. A method executed at least in part in a com...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9461433,2016-10-04,1. A light-emitting device comprising: a base;...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9229528,2016-01-05,"1. An input apparatus, comprising: a plurality...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8508147,2013-08-13,"1. A dimmer circuit, comprising: a bleeder as ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_claim_cpc_train.shape

(100, 666)

In [21]:
%%time
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

#Sentences are encoded by calling model.encode()
claim_embeddings = model.encode(df_claim_cpc_train.text[:100], convert_to_tensor=True, show_progress_bar=True)
stored_patent_train_embeddings_id = df_claim_cpc_train['id']


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 1min 53s, sys: 28.7 s, total: 2min 22s
Wall time: 2min 26s


In [11]:
def get_top_n_similar_patents_df(new_claim):
    search_hits_list = []
    new_claim_embedding = model.encode(str(new_claim), convert_to_tensor=True)
    search_hits = util.semantic_search(new_claim_embedding, claim_embeddings, 10000, 5000000, 20)
    top_claim_order = []
    top_claim_ids = []
    top_claim_text = []
    top_similarity_scores = []
    train_text_list = df_claim_cpc_train.text.to_list()
    for item in range(len(search_hits[0])):
        top_claim_order = search_hits[0][item].get('corpus_id')
        top_claim_ids.append(stored_patent_train_embeddings_id[top_claim_order])
        top_similarity_scores.append(search_hits[0][item].get('score'))
        top_claim_text.append(train_text_list[top_claim_order][3:])
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'claim':top_claim_text,
        'cosine_similarity': top_similarity_scores,
    })

        
    
    return top_100_similar_patents_df

In [13]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
#@title Patent Search using PatentSBERTa
import gradio as gr

title = "Patent Search using PatentSBERTa"

demo = gr.Interface(fn=get_top_n_similar_patents_df, inputs="text", outputs="dataframe")
    
if __name__ == "__main__":
    demo.launch()   

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>